# _**First section to put first time Github URL**_ 

## 1-Web scrape section to obtain dataframe form the specified URL

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df1=df[0]
df1.head()

##print( tabulate(df[0], headers='keys', tablefmt='psql') )
##df1=pd.DataFrame(df)
##df1.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 2-Delete rows with not assigned value in Borough from dataframe

In [2]:
df1['Borough']!='Not assigned'
df2 = df1[df1['Borough']!='Not assigned']
df2.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


## 3-Replace Borough values into Neighborhood columns with not assigned value

In [3]:
df2['Neighborhood'].replace('Not assigned', df2.Borough , inplace=True)
df2.head(10)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


## 4-Combining two rows into one row with the neighborhoods separated with a comma

In [4]:
df3=df2.groupby(['Postcode','Borough'],sort=False).agg( ','.join)

In [5]:
df3.reset_index( inplace=True)

In [6]:
df3.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


## 5-The number of rows

In [7]:
df3.shape[0]

103

# _**Second section to put second time Github URL**_ 

## 1-According to question, we can use both the Geocoder package or the csv file to create the dataframe, so I read Geospatial_data to csv

In [8]:
d = pd.read_csv('http://cocl.us/Geospatial_data')
d.rename(columns={'Postal Code':'Postal Code1'}, inplace=True)
d.head()

,Postal Code1,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## 2-Merge the two dataframes in order to create result dataframe

In [9]:
df4 = pd.merge(left=df3 , right=d, left_on='Postcode', right_on='Postal Code1').drop('Postal Code1',axis=1)
result= df4[['Postcode','Borough','Neighborhood','Latitude','Longitude']]
result.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


# _**Third section to put third time Github URL**_ 

In [12]:
# Downloading folium, if not installed
!conda install -c conda-forge folium=0.5.0 --yes
import folium # Map plotting library
import numpy as np
from pandas.io.json import json_normalize # Tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geographiclib-1.50   | 34 KB     | ##################################### | 100% 
geopy-1.20.0         | 57 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executi

### 1- Use geopy library to get the latitude and longitude values of Toronto

In [ ]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

### 2- Create a map of Toronto with neighborhoods superimposed on top

In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto 

### 3-Filter only boroughs that contain the word Toronto 

In [17]:
# filter borough names that contain the word Toronto
borough_names = list(result.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']

In [19]:
# create a new DataFrame with only boroughs that contain the word Toronto
result = result[result['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(result.shape)
result.head()

(39, 5)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [20]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto 

### 4-Use the Foursquare API to explore the neighborhoods

In [47]:
# define Foursquare Credentials and Version
CLIENT_ID = 'CLIENT_ID' # your Foursquare ID
CLIENT_SECRET = 'CLIENT_SECRET' # your Foursquare Secret
VERSION = '20200106' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CLIENT_ID
CLIENT_SECRET:CLIENT_SECRET


**Now, let's get the top 100 venues that are within a radius of 500 meters.**

In [24]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Postcode'], result['Borough'], result['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [25]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Postcode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1712, 9)


,Postcode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5A,Downtown Toronto,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,M5A,Downtown Toronto,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,Downtown Toronto,Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


**Let's check how many venues were returned for each PostCode**

In [28]:
venues_df.groupby(["Postcode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Postcode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West,Riverdale",43,43,43,43,43,43
M4L,East Toronto,"The Beaches West,India Bazaar",22,22,22,22,22,22
M4M,East Toronto,Studio District,43,43,43,43,43,43
M4N,Central Toronto,Lawrence Park,4,4,4,4,4,4
M4P,Central Toronto,Davisville North,8,8,8,8,8,8
M4R,Central Toronto,North Toronto West,17,17,17,17,17,17
M4S,Central Toronto,Davisville,32,32,32,32,32,32
M4T,Central Toronto,"Moore Park,Summerhill East",1,1,1,1,1,1


**Let's find out how many unique categories can be curated from all the returned venues**

In [29]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 234 uniques categories.


In [30]:
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Coffee Shop', 'Gym / Fitness Center', 'Spa',
       'Restaurant', 'Breakfast Spot', 'Pub', 'Historic Site', 'Park',
       'Chocolate Shop', 'Farmers Market', 'Dessert Shop',
       'Performing Arts Venue', 'Café', 'French Restaurant',
       'Mexican Restaurant', 'Event Space', 'Asian Restaurant',
       'Shoe Store', 'Cosmetics Shop', 'Ice Cream Shop', 'Theater',
       'Art Gallery', 'Brewery', 'Electronics Store', 'Bank',
       'Beer Store', 'Hotel', 'Health Food Store', 'Antique Shop',
       'Portuguese Restaurant', 'Italian Restaurant', 'Gym',
       'Sushi Restaurant', 'Creperie', 'Yoga Studio', 'Beer Bar',
       'Arts & Crafts Store', 'Burrito Place', 'Hobby Shop', 'Diner',
       'Japanese Restaurant', 'Burger Joint', 'Fried Chicken Joint',
       'Wings Joint', 'Nightclub', 'Chinese Restaurant', 'Smoothie Shop',
       'Sandwich Place', 'College Auditorium'], dtype=object)

### 5- Analyze Each Area

In [32]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['Postcode'] = venues_df['Postcode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1712, 237)


,Postcode,Borough,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M5A,Downtown Toronto,Harbourfront,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,Harbourfront,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,Downtown Toronto,Harbourfront,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,Downtown Toronto,Harbourfront,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,Downtown Toronto,Harbourfront,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [34]:
toronto_grouped = toronto_onehot.groupby(["Postcode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 237)


,Postcode,Borough,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,M4K,East Toronto,"The Danforth West,Riverdale",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.023256,...,0.023256,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.023256
2,M4L,East Toronto,"The Beaches West,India Bazaar",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,M4M,East Toronto,Studio District,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.046512,...,0.000000,0.00,0.000000,0.000000,0.000000,0.023256,0.000000,0.000000,0.00,0.023256
4,M4N,Central Toronto,Lawrence Park,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,M4P,Central Toronto,Davisville North,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,M4R,Central Toronto,North Toronto West,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.058824
7,M4S,Central Toronto,Davisville,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,M4T,Central Toronto,"Moore Park,Summerhill East",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.066667,...,0.000000,0.00,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.00,0.000000


Now let's create the new dataframe and display the top 10 venues for each PostalCode.

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['Postcode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = toronto_grouped['Postcode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,Postcode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Trail,Neighborhood,Health Food Store,Pub,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Bookstore,Furniture / Home Store,Bubble Tea Shop,Spa,Juice Bar
2,M4L,East Toronto,"The Beaches West,India Bazaar",Park,Sandwich Place,Pizza Place,Pub,Burger Joint,Burrito Place,Liquor Store,Fast Food Restaurant,Italian Restaurant,Fish & Chips Shop
3,M4M,East Toronto,Studio District,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Italian Restaurant,Gastropub,Yoga Studio,Fish Market,Pet Store
4,M4N,Central Toronto,Lawrence Park,Park,Lawyer,Bus Line,Swim School,Diner,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
5,M4P,Central Toronto,Davisville North,Gym,Clothing Store,Park,Food & Drink Shop,Sandwich Place,Hotel,Dance Studio,Breakfast Spot,Cuban Restaurant,Creperie
6,M4R,Central Toronto,North Toronto West,Sporting Goods Shop,Coffee Shop,Yoga Studio,Burger Joint,Café,Chinese Restaurant,Clothing Store,Dessert Shop,Diner,Mexican Restaurant
7,M4S,Central Toronto,Davisville,Dessert Shop,Sandwich Place,Pizza Place,Coffee Shop,Sushi Restaurant,Gym,Italian Restaurant,Café,Indie Movie Theater,Diner
8,M4T,Central Toronto,"Moore Park,Summerhill East",Playground,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",Coffee Shop,Pub,Pizza Place,Light Rail Station,Liquor Store,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Fried Chicken Joint


### 6- Cluster Areas
Run k-means to cluster the Toronto areas into 5 clusters.

In [36]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["Postcode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 1, 1, 1, 1, 1, 3, 1], dtype=int32)

In [38]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = result.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("Postcode"), on="Postcode")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(39, 16)


,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Mexican Restaurant,Performing Arts Venue,Brewery,Shoe Store
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,1,Coffee Shop,Park,Sushi Restaurant,Gym,College Cafeteria,Burger Joint,Sandwich Place,Burrito Place,Café,Chinese Restaurant
2,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Theater,Tea Room,Fast Food Restaurant,Restaurant,Japanese Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Restaurant,Hotel,Breakfast Spot,Clothing Store,Beer Bar,Cosmetics Shop,Bakery,Park
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Neighborhood,Health Food Store,Pub,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [39]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Mexican Restaurant,Performing Arts Venue,Brewery,Shoe Store
20,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Gym,Clothing Store,Park,Food & Drink Shop,Sandwich Place,Hotel,Dance Studio,Breakfast Spot,Cuban Restaurant,Creperie
21,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307,1,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
24,M5R,Central Toronto,"The Annex,North Midtown,Yorkville",43.672710,-79.405678,1,Sandwich Place,Café,Coffee Shop,Park,Pub,BBQ Joint,History Museum,Indian Restaurant,Pharmacy,Pizza Place
25,M6R,West Toronto,"Parkdale,Roncesvalles",43.648960,-79.456325,1,Gift Shop,Bookstore,Dessert Shop,Eastern European Restaurant,Italian Restaurant,Bar,Dog Run,Restaurant,Movie Theater,Breakfast Spot
26,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Dessert Shop,Sandwich Place,Pizza Place,Coffee Shop,Sushi Restaurant,Gym,Italian Restaurant,Café,Indie Movie Theater,Diner
27,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049,1,Café,Restaurant,Japanese Restaurant,Bookstore,Italian Restaurant,Bar,Sandwich Place,Bakery,College Arts Building,Dessert Shop
28,M6S,West Toronto,"Runnymede,Swansea",43.651571,-79.484450,1,Café,Pizza Place,Sushi Restaurant,Coffee Shop,Italian Restaurant,Fish & Chips Shop,Bar,Food,Diner,Indie Movie Theater
29,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,1,Playground,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
30,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market",43.653206,-79.400049,1,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Dumpling Restaurant,Coffee Shop,Bar,Mexican Restaurant,Bakery,Tea Room


**Finally, let's visualize the resulting clusters**

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postcode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 7-Examine Clusters

#### Cluster 1

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Mexican Restaurant,Performing Arts Venue,Brewery,Shoe Store


#### Cluster 2

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Central Toronto,1,Gym,Clothing Store,Park,Food & Drink Shop,Sandwich Place,Hotel,Dance Studio,Breakfast Spot,Cuban Restaurant,Creperie
21,Central Toronto,1,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
24,Central Toronto,1,Sandwich Place,Café,Coffee Shop,Park,Pub,BBQ Joint,History Museum,Indian Restaurant,Pharmacy,Pizza Place
25,West Toronto,1,Gift Shop,Bookstore,Dessert Shop,Eastern European Restaurant,Italian Restaurant,Bar,Dog Run,Restaurant,Movie Theater,Breakfast Spot
26,Central Toronto,1,Dessert Shop,Sandwich Place,Pizza Place,Coffee Shop,Sushi Restaurant,Gym,Italian Restaurant,Café,Indie Movie Theater,Diner
27,Downtown Toronto,1,Café,Restaurant,Japanese Restaurant,Bookstore,Italian Restaurant,Bar,Sandwich Place,Bakery,College Arts Building,Dessert Shop
28,West Toronto,1,Café,Pizza Place,Sushi Restaurant,Coffee Shop,Italian Restaurant,Fish & Chips Shop,Bar,Food,Diner,Indie Movie Theater
29,Central Toronto,1,Playground,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
30,Downtown Toronto,1,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Dumpling Restaurant,Coffee Shop,Bar,Mexican Restaurant,Bakery,Tea Room
31,Central Toronto,1,Coffee Shop,Pub,Pizza Place,Light Rail Station,Liquor Store,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Fried Chicken Joint


#### Cluster 3

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,West Toronto,2,Mexican Restaurant,Café,Thai Restaurant,Grocery Store,Fried Chicken Joint,Park,Music Venue,Diner,Cajun / Creole Restaurant,Fast Food Restaurant


#### Cluster 4

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Downtown Toronto,3,Coffee Shop,Bar,Steakhouse,Café,Hotel,Sushi Restaurant,Asian Restaurant,Thai Restaurant,Restaurant,Burger Joint


#### Cluster 5

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,4,Sporting Goods Shop,Coffee Shop,Yoga Studio,Burger Joint,Café,Chinese Restaurant,Clothing Store,Dessert Shop,Diner,Mexican Restaurant
10,Downtown Toronto,4,Coffee Shop,Aquarium,Café,Italian Restaurant,Hotel,Fried Chicken Joint,Restaurant,Scenic Lookout,Brewery,Bakery
